
**Run a Minecraft Server on Google Colab!**

---

# Start the server

The script below will start your server. You'll have to create a server first to be able to use it.

In [ ]:
import os
import re
import json
from google.colab import drive

# Check java version
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

if java_ver[0] == "11" :
  # Update package list
  !sudo apt update &>/dev/null && echo "apt updated correctly" || echo "error updating apt"

  # Remove java 11
  !sudo apt remove -y openjdk-11-jdk-headless openjdk-11-jre-headless &>/dev/null && echo "java 11 removed correctly" || echo "error removing java 11"
  !sudo apt autoremove -y &>/dev/null && echo "packages cleaned correctly" || echo "error cleaning packages"
  
  # Install java 21
  !sudo apt install -y wget apt-transport-https &>/dev/null && echo "installed wget and apt-transport-https" || echo "error installing wget and apt-transport-https"
  !mkdir -p /etc/apt/keyrings &>/dev/null && echo "created key directory" || echo "error creating key directory"
  !wget -O - https://packages.adoptium.net/artifactory/api/gpg/key/public | tee /etc/apt/keyrings/adoptium.asc &>/dev/null && echo "obtaining keys" || echo "error obtaining keys"
  !echo "deb [signed-by=/etc/apt/keyrings/adoptium.asc] https://packages.adoptium.net/artifactory/deb $(awk -F= '/^VERSION_CODENAME/{print$2}' /etc/os-release) main" | tee /etc/apt/sources.list.d/adoptium.list &>/dev/null && echo "keys added" || echo "error adding keys"
  !sudo apt update &>/dev/null && echo "apt updated correctly" || echo "error updating apt"
  !sudo apt install temurin-21-jdk &>/dev/null && echo "java 21 installed correctly." || echo "error installing java 21."
  
  # Clean package list
  !sudo apt clean &>/dev/null && echo "packages cleaned correctly" || echo "error cleaning packages"
  
  # Check java version
  java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

if java_ver[0] == "21" :
  print("java 21 is working correctly, you can continue.")

  # Mount Google Drive
  drive.mount('/content/drive')
  
  # Change directory
  %cd "/content/drive/My Drive/minecraft-server"

  !ls #list directory content

  # Import configuration file.
  if os.path.isfile("colabconfig.json"):
    colabconfig = json.load(open("colabconfig.json"))
  else:
    colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
    json.dump(colabconfig, open("colabconfig.json",'w'))

  # Server jar names.
  jar_list = {'paper': 'server.jar', 'purpur': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
  jar_name = jar_list[colabconfig["server_type"]]

  # Java arguments.
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
  memory_allocation = "-Xms6G -Xmx6G"

  # Choose tunnel service
  # Available Options: ngrok, playit, none
  tunnel_service = "ngrok"
  print("Using", tunnel_service)
  
  if tunnel_service == "ngrok":
    !pip -q install pyngrok
    from pyngrok import conf, ngrok

    # Ngrok token
    print("Obtain your token from https://dashboard.ngrok.com/get-started/your-authtoken")
    import getpass
    authtoken = getpass.getpass() # Set Ngrok token every start or replace it
    !ngrok authtoken $authtoken # Ngrok login

    # Set Ngrok region
    conf.get_default().region = 'sa' # Change this to region you want, in this case is Sudamerica

    # Connect to Ngrok
    url = ngrok.connect(25565, 'tcp')
    print('Your server direction is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
    print('Starting server...')
    !java $memory_allocation $server_flags -jar $jar_name nogui

  elif tunnel_service == "playit":
    !curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg >/dev/null
    !echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list
    !sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "error installing Playit.gg"
    print('Starting server...')
    !playit & java $memory_allocation $server_flags -jar $jar_name nogui
  
  elif tunnel_service == "none":
    print('Starting server...')
    !java $memory_allocation $server_flags -jar $jar_name nogui

  else:
    print('Tunnel service not valid')
else:
  print("java 21 is not installed or is not working")

# Make a server

The script below will download a server and accept the EULA. Then you can start it.

In [ ]:
from google.colab import drive
import requests
import json

# Replace "1.20.4" with version you want.
# Available versions:
# - 1.20.4
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# New versions maybe work, but is not guaranteed.
version = '1.20.4'

# Choose server type. Available versions: paper, purpur, fabric
server_type = 'purpur'

drive.mount('/content/drive')
!mkdir "/content/drive/My Drive/minecraft-server"
%cd "/content/drive/My Drive/minecraft-server"

if server_type == "paper":
  a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
  b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
  url = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]
  jar_name = "server.jar"

elif server_type == "purpur":
  url = f"https://api.purpurmc.org/v2/purpur/{version}/latest/download"
  jar_name = "server.jar"

elif server_type == "fabric":
  url = "https://maven.fabricmc.net/net/fabricmc/fabric-installer/1.0.1/fabric-installer-1.0.1.jar"
  jar_name = "fabric-installer.jar"

print('Downloading to Google Drive...')
r = requests.get(url)
if r.status_code == 200:
  with open('/content/drive/My Drive/minecraft-server/' + jar_name, 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + '! It seems that you choose an unsupported version. Try running the script again.')

if server_type == "fabric":
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

colabconfig = {"server_type": server_type}
json.dump(colabconfig, open("colabconfig.json",'w'))

# Accept the EULA
%cd "/content/drive/My Drive/minecraft-server"
!echo "eula=true" >> eula.txt

print('Ready!')